Install **kubectl** and the **Google Cloud SDK** with the necessary authentication plugin for Google Kubernetes Engine (GKE).

In [ ]:
%%bash

curl -LO "https://dl.k8s.io/release/$(curl -L -s https://dl.k8s.io/release/stable.txt)/bin/linux/amd64/kubectl"
sudo install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl
apt-get update && apt-get install apt-transport-https ca-certificates gnupg
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo gpg --dearmor -o /usr/share/keyrings/cloud.google.gpg
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee -a /etc/apt/sources.list.d/google-cloud-sdk.list
apt-get update && sudo apt-get install google-cloud-cli-gke-gcloud-auth-plugin


**Replace** <CLUSTER_NAME> with your cluster name, e.g. postgres-cluster. Retrieve the GKE cluster's credentials using the **gcloud** command:

In [ ]:
%%bash

export KUBERNETES_CLUSTER_NAME=<CLUSTER_NAME>
gcloud container clusters get-credentials $KUBERNETES_CLUSTER_NAME --region $GOOGLE_CLOUD_REGION

Create an .env file with environment variables required for connecting to Postgresql and LLM runtime in a Kubernetes cluster.

In [ ]:
%%bash

echo POSTGRES_ENDPOINT=$(kubectl get pod -l spilo-role=master -n postgres -o=jsonpath="{.items[0].status.podIP}") > .env
echo LLM_ENDPOINT=http://$(kubectl get pod -l app=tgi-runtime -n llm -o=jsonpath="{.items[0].status.podIP}"):8000 >> .env
echo DATABASE_NAME=mydatabase >> .env
echo DBUSERNAME=$(kubectl get secret mydatabaseowner.my-cluster.credentials.postgresql.acid.zalan.do -n postgres --template={{.data.username}} | base64 -d) >> .env
echo DBPASSWORD=$(kubectl get secret mydatabaseowner.my-cluster.credentials.postgresql.acid.zalan.do -n postgres --template={{.data.password}} | base64 -d) >> .env

Install required python libraries:

In [ ]:
! pip install python-dotenv psycopg-binary psycopg tabulate text-generation langchain langchain-community

Import python libraries:

In [70]:
from dotenv import load_dotenv
import psycopg
import os
from tabulate import tabulate
from langchain_community.llms import HuggingFaceTextGenInference
from langchain_core.prompts import PromptTemplate

Load environment variables from the .env file, establish a connection to a PostgreSQL database and retrieve information about the schema of the public tables.

In [71]:
load_dotenv()

conn = psycopg.connect(
    dbname=os.environ.get("DATABASE_NAME"),
    host=os.environ.get("POSTGRES_ENDPOINT"),
    user=os.environ.get("DBUSERNAME"),
    password=os.environ.get("DBPASSWORD"),
    autocommit=True)

db_schema = conn.execute("SELECT table_name, column_name as Columns, data_type as DataTypes FROM  information_schema.columns where table_name NOT LIKE 'pg_stat%' AND table_schema='public' order by table_name,column_name;")
colnames = [desc[0] for desc in db_schema.description]
db_schema_formatted=tabulate(db_schema.fetchall(), headers=colnames, tablefmt='psql')


Initializes the Hugging Face TGI connection for text generation.

Set up two prompts: the first one is for generating SQL commands based on user queries, while the second prompt is for generating responses based on user queries and PostgreSQL replies.

In [74]:
llm = HuggingFaceTextGenInference(
    inference_server_url=os.environ.get("LLM_ENDPOINT"),
    temperature=0.5,
    top_k=5,
    top_p=0.5,
    repetition_penalty=1.03,
)

sql_prompt_template = PromptTemplate.from_template("""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful AI assistant that can transform user queries into SQL commands to retrieve the data from the Postgresql database. The database has the next tables schema:
    {db_schema}
    Please prepare and return only the SQL command, based on the user query, without any formatting or newlines. The answer must contain only valid SQL command.<|eot_id|><|start_header_id|>user<|end_header_id|>
    {query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")

final_prompt_template = PromptTemplate.from_template("""
    <|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a helpful AI assistant that can understand Postgresql replies and explain this data to the user. The database has the next tables schema:
    {db_schema}
    User query: {query}
    Postgresql reply:
    {postgres_reply}
    Base your answer on the provided user query and Postgresql reply.
    Generate a draft response using the selected information.
    It should be easy to understand your answer. Don't add any introductory words, start answering right away.
    Keep your answer to a one or two sentences (if possible) that specifically answers the user's question. If not - try to keep the answer short, summarizing the returned data.
    Generate your final response after adjusting it to increase accuracy and relevance.
    Now only show your final response!
    If you do not know the answer or context is not relevant, response with "I don't know".
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
""")

Configure two functions to interacte with the PostgreSQL database and the TGI runtime.

In [81]:
def postgres_query(query):
    try:
        postgres_reply = conn.execute(query)
    except psycopg.Error as e:
        print("Unable to process query")
        return False
    colnames = [desc[0] for desc in postgres_reply.description]
    postgres_reply_data = postgres_reply.fetchall()
    if postgres_reply_data == []:
        print("Received empty SQL reply")
        return False
    postgres_reply_formatted=tabulate(postgres_reply_data, headers=colnames, tablefmt='psql')
    return postgres_reply_formatted


def llm_query(query):
    sql_prompt_value=sql_prompt_template.format(db_schema=db_schema_formatted, query=query)
    #print(sql_prompt_value)
    #print("-----")
    sql_query=llm.invoke(sql_prompt_value)
    #print(sql_query)
    #print("-----")
    postgres_reply=postgres_query(sql_query)
    if postgres_reply == False:
        return "Try another query"
    #print(postgres_reply)
    #print("-----")
    final_prompt_value=final_prompt_template.format(db_schema=db_schema_formatted, query=query, postgres_reply=postgres_reply)
    #print(final_prompt_value)
    #print("----")
    return llm.invoke(final_prompt_value)

Run some queries to demonstrate the model's ability to generate SQL commands from user queries and provide responses based on the PostgreSQL replies:

In [ ]:
print(llm_query("Please calculate the total sum of all John transactions."))
print(llm_query("Which woman spent more money in 2023 and how much?"))
print(llm_query("What is the capital of Great Britain?"))
print(llm_query("Who spent more money on electronics in last month?"))
print(llm_query("Who spent more money on electronics in last year?"))
print(llm_query("Give me top 3 buyers of clothing. How much money each person spent?"))